<a href="https://colab.research.google.com/github/anmol8227/Fake-News-Detiction/blob/main/fake_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls

fake_or_real_news.csv  sample_data


importing the dependencies

In [2]:
import pandas as pd
import pandas as np
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
# printing stopwords in english
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data pre-Processing

In [5]:
#loading the dataset to a pandas dataFrame
news_dataset =pd.read_csv('/content/fake_or_real_news.csv')

In [6]:
news_dataset.shape

(6335, 4)

In [7]:
#print the first 5 rows of the dataframe
news_dataset.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [8]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

,0
Unnamed: 0,0
title,0
text,0
label,0


In [9]:
news_dataset = news_dataset.fillna('')

In [10]:
news_dataset['title']= news_dataset['text']+ ' ' +news_dataset['label']

In [11]:
print(news_dataset['title'])

0       Daniel Greenfield, a Shillman Journalism Fello...
1       Google Pinterest Digg Linkedin Reddit Stumbleu...
2       U.S. Secretary of State John F. Kerry said Mon...
3       — Kaydee King (@KaydeeKing) November 9, 2016 T...
4       It's primary day in New York and front-runners...
                              ...                        
6330    The State Department told the Republican Natio...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332     Anti-Trump Protesters Are Tools of the Oligar...
6333    ADDIS ABABA, Ethiopia —President Obama convene...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: title, Length: 6335, dtype: object


Stemming
stemming is the process of reducing a world its root word

In [12]:
port_stem = PorterStemmer()

In [13]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content= [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [14]:
news_dataset['title'] = news_dataset['title'].apply(stemming)

In [15]:
print(news_dataset['title'])

0       daniel greenfield shillman journal fellow free...
1       googl pinterest digg linkedin reddit stumbleup...
2       u secretari state john f kerri said monday sto...
3       kayde king kaydeek novemb lesson tonight dem l...
4       primari day new york front runner hillari clin...
                              ...                        
6330    state depart told republican nation committe c...
6331    p pb stand plutocrat pentagon post oct wikimed...
6332    anti trump protest tool oligarchi reform alway...
6333    addi ababa ethiopia presid obama conven meet l...
6334    jeb bush suddenli attack trump matter jeb bush...
Name: title, Length: 6335, dtype: object


In [16]:
#separiting the data & subject
X = news_dataset['title'].values
Y = news_dataset['label'].values

In [17]:
print(X)

['daniel greenfield shillman journal fellow freedom center new york writer focus radic islam final stretch elect hillari rodham clinton gone war fbi word unpreced thrown around often elect ought retir still unpreced nomine major polit parti go war fbi exactli hillari peopl done coma patient wake watch hour cnn hospit bed would assum fbi director jame comey hillari oppon elect fbi attack everyon obama cnn hillari peopl circul letter attack comey current media hit piec lambast target trump surpris clinton alli start run attack ad fbi fbi leadership warn entir left wing establish form lynch mob continu go hillari fbi credibl attack media democrat preemptiv head result investig clinton foundat hillari clinton covert struggl fbi agent obama doj peopl gone explos public new york time compar comey j edgar hoover bizarr headlin jame comey role recal hoover fbi fairli practic admit front spout nonsens boston globe publish column call comey resign outdon time editori claim scandal realli attack 

In [18]:
print(Y)

['FAKE' 'FAKE' 'REAL' ... 'FAKE' 'REAL' 'REAL']


In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

In [20]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(5068,)
(1267,)
(5068,)
(1267,)


In [21]:
#converting the textual data to numerical dat
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [22]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1604141 stored elements and shape (6335, 43733)>
  Coords	Values
  (0, 106)	0.02761223178968733
  (0, 212)	0.02332459017101427
  (0, 271)	0.05439002473173256
  (0, 328)	0.06035554077191256
  (0, 357)	0.013921109035269065
  (0, 451)	0.020577381150870933
  (0, 578)	0.07905692177246273
  (0, 617)	0.019849968559473807
  (0, 620)	0.018676826688988558
  (0, 622)	0.04460898268244669
  (0, 648)	0.015492864876614556
  (0, 680)	0.019223803689908937
  (0, 978)	0.026730386870274004
  (0, 999)	0.02073715241383108
  (0, 1022)	0.037146876603784226
  (0, 1033)	0.014808004202970843
  (0, 1085)	0.02881367704604006
  (0, 1089)	0.018101578316216945
  (0, 1197)	0.022504651760450495
  (0, 1206)	0.010675016595249835
  (0, 1549)	0.025075776999905885
  (0, 1665)	0.025966498055121984
  (0, 1713)	0.04112619145448393
  (0, 1744)	0.027617282842453923
  (0, 1783)	0.018871365842945256
  :	:
  (6334, 39650)	0.023983405159182066
  (6334, 39971)	0.0407153638

Splitting the dataset to training & test data

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size =0.2,stratify=Y,random_state=2)

Training the model:Logistic Regression

In [24]:
model = LogisticRegression()

In [25]:
model = LogisticRegression()

Evaluation

accuracy score

In [26]:
# accuracy score on the training data
model.fit(X_train, Y_train)
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [27]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('accuracy score of the test data :',test_data_accuracy)

accuracy score of the test data : 0.9368587213891081


Making a Predictive System

In [28]:
X_new = X_test[1]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]=='1'):
  print('the news is Real')
else:
  print('the news is Fake')

['FAKE']
the news is Fake


In [29]:
print(Y_test[0])

REAL


In [30]:
print(Y_test[2])

FAKE


# Task
Explain the error in the selected code. If possible, fix the error and incorporate the changes into the existing code. Otherwise, try to diagnose the error. Also, add more evaluation metrics, try a different model, and build a prediction function.

## Add more evaluation metrics

### Subtask:
Calculate and display precision, recall, F1-score, and a confusion matrix for the current Logistic Regression model.


**Reasoning**:
Import necessary metrics from sklearn and calculate precision, recall, F1-score, and the confusion matrix for the test data.



In [31]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

precision = precision_score(Y_test, X_test_prediction, pos_label='FAKE')
recall = recall_score(Y_test, X_test_prediction, pos_label='FAKE')
f1 = f1_score(Y_test, X_test_prediction, pos_label='FAKE')
conf_matrix = confusion_matrix(Y_test, X_test_prediction, labels=['FAKE', 'REAL'])

print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)
print('Confusion Matrix:\n', conf_matrix)

Precision: 0.9247311827956989
Recall: 0.9510268562401264
F1-score: 0.9376947040498442
Confusion Matrix:
 [[602  31]
 [ 49 585]]


 different model

### Subtask:
Implement another classification model, such as Naive Bayes or Support Vector Machines, and evaluate its performance.


**Reasoning**:
Implement and evaluate a Multinomial Naive Bayes model as an alternative classification model.



In [32]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Instantiate the Multinomial Naive Bayes model
new_model = MultinomialNB()

# Train the new model
new_model.fit(X_train, Y_train)

# Make predictions on the test data
X_test_prediction_new_model = new_model.predict(X_test)

# Calculate and print the accuracy score
test_data_accuracy_new_model = accuracy_score(X_test_prediction_new_model, Y_test)
print('Accuracy score of the Multinomial Naive Bayes model on the test data:', test_data_accuracy_new_model)

# Calculate and print precision, recall, and F1-score
precision_new_model = precision_score(Y_test, X_test_prediction_new_model, pos_label='FAKE')
recall_new_model = recall_score(Y_test, X_test_prediction_new_model, pos_label='FAKE')
f1_new_model = f1_score(Y_test, X_test_prediction_new_model, pos_label='FAKE')

print('Precision (Multinomial Naive Bayes):', precision_new_model)
print('Recall (Multinomial Naive Bayes):', recall_new_model)
print('F1-score (Multinomial Naive Bayes):', f1_new_model)

# Calculate and print the confusion matrix
conf_matrix_new_model = confusion_matrix(Y_test, X_test_prediction_new_model, labels=['FAKE', 'REAL'])
print('Confusion Matrix (Multinomial Naive Bayes):\n', conf_matrix_new_model)

Accuracy score of the Multinomial Naive Bayes model on the test data: 0.8468823993685872
Precision (Multinomial Naive Bayes): 0.9660297239915074
Recall (Multinomial Naive Bayes): 0.7187993680884676
F1-score (Multinomial Naive Bayes): 0.8242753623188406
Confusion Matrix (Multinomial Naive Bayes):
 [[455 178]
 [ 16 618]]


## Build a prediction function

### Subtask:
Create a function that takes raw text input and outputs a prediction using the trained model.


**Reasoning**:
Define a function to predict news authenticity based on raw text input by applying the same preprocessing steps and using the trained Logistic Regression model.



In [33]:
def predict_news(raw_text):
    # Clean the text using the stemming function
    cleaned_text = stemming(raw_text)
    # Transform the cleaned text using the fitted vectorizer
    transformed_text = vectorizer.transform([cleaned_text])
    # Make a prediction using the trained model
    prediction = model.predict(transformed_text)

    # Return a user-friendly string based on the prediction
    if (prediction[0] == 'REAL'):
        return 'The news is Real'
    else:
        return 'The news is Fake'


In [34]:
print(Y_test[0])

REAL


In [35]:
print(Y_test[2])

FAKE


# Task
Explain the error in the selected code, fix it if possible, and then add the following enhancements: more evaluation metrics (precision, recall, F1-score, confusion matrix), another classification model (Naive Bayes or SVM), hyperparameter tuning, advanced text preprocessing (lemmatization or n-grams), visualizations (word clouds, performance plots), a prediction function, and model saving/loading. Explain the added components and results.

## Perform hyperparameter tuning

### Subtask:
Tune the hyperparameters of the Logistic Regression and the other implemented model(s) to potentially improve their performance.


**Reasoning**:
Import GridSearchCV and define parameter grids for Logistic Regression and Multinomial Naive Bayes, then perform GridSearchCV for both models to find the best hyperparameters.



In [36]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid for Logistic Regression
param_grid_lr = {'C': [0.1, 1, 10, 100]}

# Instantiate GridSearchCV for Logistic Regression
grid_search_lr = GridSearchCV(LogisticRegression(), param_grid_lr, cv=5)

# Fit GridSearchCV for Logistic Regression
grid_search_lr.fit(X_train, Y_train)

# Print best hyperparameters for Logistic Regr7ession
print("Best hyperparameters for Logistic Regression:", grid_search_lr.best_params_)

# Define parameter grid for Multinomial Naive Bayes
param_grid_nb = {'alpha': [0.1, 0.5, 1.0]}

# Instantiate GridSearchCV for Multinomial Naive Bayes
grid_search_nb = GridSearchCV(MultinomialNB(), param_grid_nb, cv=5)

# Fit GridSearchCV for Multinomial Naive Bayes
grid_search_nb.fit(X_train, Y_train)

# Print best hyperparameters for Multinomial Naive Bayes
print("Best hyperparameters for Multinomial Naive Bayes:", grid_search_nb.best_params_)

Best hyperparameters for Logistic Regression: {'C': 100}
Best hyperparameters for Multinomial Naive Bayes: {'alpha': 0.1}


## Implement advanced text preprocessing

### Subtask:
Incorporate techniques like lemmatization or using n-grams into the text preprocessing step.


**Reasoning**:
Incorporate n-grams into the TF-IDF vectorization.



In [37]:
# Re-vectorize the data using n-grams
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
vectorizer.fit(news_dataset['title'])

X = vectorizer.transform(news_dataset['title'])

# Split the data again with the new feature vectors
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

print(X_train.shape)
print(X_test.shape)

(5068, 1397832)
(1267, 1397832)


## Add visualizations

### Subtask:
Create visualizations to explore the data and model results, such as word clouds or plots of model performance.


**Reasoning**:
Generate word clouds for fake and real news titles to visualize the most frequent words in each category.



In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Separate fake and real news titles
fake_news_titles = news_dataset[news_dataset['label'] == 'FAKE']['title']
real_news_titles = news_dataset[news_dataset['label'] == 'REAL']['title']

# Generate word cloud for fake news
fake_wordcloud = WordCloud(width=800, height=400, random_state=21, max_words=100, stopwords=stopwords.words('english')).generate(" ".join(fake_news_titles))

# Generate word cloud for real news
real_wordcloud = WordCloud(width=800, height=400, random_state=21, max_words=100, stopwords=stopwords.words('english')).generate(" ".join(real_news_titles))

# Plot the word clouds
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(fake_wordcloud, interpolation="bilinear")
plt.axis('off')
plt.title('Word Cloud for Fake News Titles')

plt.subplot(1, 2, 2)
plt.imshow(real_wordcloud, interpolation="bilinear")
plt.axis('off')
plt.title('Word Cloud for Real News Titles')

plt.tight_layout()
plt.show()

**Reasoning**:
Create a bar plot to compare the performance metrics (accuracy, precision, recall, F1-score) of the Logistic Regression and Multinomial Naive Bayes models.



In [ ]:
import numpy as np

# Re-train models with best hyperparameters found
best_lr_model = LogisticRegression(C=grid_search_lr.best_params_['C'])
best_lr_model.fit(X_train, Y_train)

best_nb_model = MultinomialNB(alpha=grid_search_nb.best_params_['alpha'])
best_nb_model.fit(X_train, Y_train)

# Make predictions with the best models
X_test_prediction_lr = best_lr_model.predict(X_test)
X_test_prediction_nb = best_nb_model.predict(X_test)

# Calculate metrics for Logistic Regression
accuracy_lr = accuracy_score(Y_test, X_test_prediction_lr)
precision_lr = precision_score(Y_test, X_test_prediction_lr, pos_label='FAKE')
recall_lr = recall_score(Y_test, X_test_prediction_lr, pos_label='FAKE')
f1_lr = f1_score(Y_test, X_test_prediction_lr, pos_label='FAKE')

# Calculate metrics for Multinomial Naive Bayes
accuracy_nb = accuracy_score(Y_test, X_test_prediction_nb)
precision_nb = precision_score(Y_test, X_test_prediction_nb, pos_label='FAKE')
recall_nb = recall_score(Y_test, X_test_prediction_nb, pos_label='FAKE')
f1_nb = f1_score(Y_test, X_test_prediction_nb, pos_label='FAKE')

# Create a bar plot
labels = ['Accuracy', 'Precision', 'Recall', 'F1-score']
lr_metrics = [accuracy_lr, precision_lr, recall_lr, f1_lr]
nb_metrics = [accuracy_nb, precision_nb, recall_nb, f1_nb]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(x - width/2, lr_metrics, width, label='Logistic Regression')
rects2 = ax.bar(x + width/2, nb_metrics, width, label='Multinomial Naive Bayes')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Score')
ax.set_title('Model Performance Comparison')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('%.2f' % height,
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)

fig.tight_layout()

plt.show()

## Build a prediction function

### Subtask:
Create a function that takes raw text input and outputs a prediction using the trained model.


**Reasoning**:
Define a function to predict news authenticity based on raw text input by applying the same preprocessing steps and using the trained Logistic Regression model.



In [ ]:
def predict_news(raw_text):
    # Clean the text using the stemming function
    cleaned_text = stemming(raw_text)
    # Transform the cleaned text using the fitted vectorizer
    transformed_text = vectorizer.transform([cleaned_text])
    # Make a prediction using the trained model
    prediction = best_lr_model.predict(transformed_text)

    # Return a user-friendly string based on the prediction
    if (prediction[0] == 'REAL'):
        return 'The news is Real'
    else:
        return 'The news is Fake'


**Reasoning**:
Test the predict_news function with an example input.



In [ ]:
# Example usage of the predict_news function
example_text = "The president is going to visit the moon next week."
prediction_result = predict_news(example_text)
print(prediction_result)

## Save and load the model

### Subtask:
Add code to save the trained model and demonstrate how to load it.


**Reasoning**:
Import joblib and save the trained Logistic Regression model to a file, then load it back to demonstrate saving and loading.



In [ ]:
import joblib

# Define filename for the model
filename = 'logistic_regression_model.joblib'

# Save the trained model to the file
joblib.dump(best_lr_model, filename)

# Load the model from the file
loaded_model = joblib.load(filename)

# Optional: Verify loaded model by making a prediction
sample_text_transformed = vectorizer.transform(["this is a test piece of text"])
original_prediction = best_lr_model.predict(sample_text_transformed)
loaded_prediction = loaded_model.predict(sample_text_transformed)

print(f"Original model prediction: {original_prediction}")
print(f"Loaded model prediction: {loaded_prediction}")

# Verify they are the same
print(f"Predictions match: {original_prediction[0] == loaded_prediction[0]}")

# Task
Enhance the text classification notebook by adding more classification models, exploring different text vectorization techniques, performing detailed error analysis, implementing cross-validation, visualizing confusion matrices, and building a simple interactive demo. Explain all added components and results.

## Implement additional classification models

### Subtask:
Add code to implement and evaluate more classification models such as SVM and a basic deep learning model if feasible.


**Reasoning**:
Import necessary libraries, instantiate, train, and evaluate the SVC model with accuracy, precision, recall, F1-score, and confusion matrix.



In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Instantiate the Support Vector Machine model
svm_model = SVC(kernel='linear')

# Train the SVM model
svm_model.fit(X_train, Y_train)

# Make predictions on the test data
X_test_prediction_svm = svm_model.predict(X_test)

# Calculate and print the accuracy score
test_data_accuracy_svm = accuracy_score(X_test_prediction_svm, Y_test)
print('Accuracy score of the SVM model on the test data:', test_data_accuracy_svm)

# Calculate and print precision, recall, and F1-score
precision_svm = precision_score(Y_test, X_test_prediction_svm, pos_label='FAKE')
recall_svm = recall_score(Y_test, X_test_prediction_svm, pos_label='FAKE')
f1_svm = f1_score(Y_test, X_test_prediction_svm, pos_label='FAKE')

print('Precision (SVM):', precision_svm)
print('Recall (SVM):', recall_svm)
print('F1-score (SVM):', f1_svm)

# Calculate and print the confusion matrix
conf_matrix_svm = confusion_matrix(Y_test, X_test_prediction_svm, labels=['FAKE', 'REAL'])
print('Confusion Matrix (SVM):\n', conf_matrix_svm)